In [3]:
import pandas as pd
tables = pd.read_html('https://trucksuvidha.com/')
df = tables[0]
df.head()

,Source City,Destination City,Distance,Weight (MT),Schd. Date,Material,Truck Type,Posted By,Details,Quotation
0,Bangalore (KA),Noida (UP),"2,130 km",Upto 12 MT,26-Feb-2025,Others,Truck 15MT / 10 w...,Transporter,View Contact,Quote Now
1,Jamui (BR),Noida (UP),"1,180 km",Upto 12 MT,26-Feb-2025,Others,Truck 20MT / 12 w...,Transporter,View Contact,Quote Now
2,Jamui (BR),Nagpur (MH),"1,126 km",Do Not Know,26-Feb-2025,Others,Canters 17feet / ...,Transporter,View Contact,Quote Now
3,Nagpur (MH),Hisar (HR),"1,302 km",Do Not Know,26-Feb-2025,Packed Food,Canters 19feet / ...,Transporter,View Contact,Quote Now
4,Bhagabannagar (TR),Hyd (AP),"2,754 km",Do Not Know,26-Feb-2025,Others,Canters 19feet / ...,Transporter,View Contact,Quote Now


In [2]:
import pandas as pd
import datetime
import os

def collect_and_store_data():
    #"""Fetches data from the website and stores it with a timestamp."""
    tables = pd.read_html('https://trucksuvidha.com/')
    df = tables[0]
    # Get current timestamp
    timestamp = datetime.datetime.now().strftime("%Y/%m/%d_%H:%M:%S")    
    # Add timestamp column to DataFrame
    df['timestamp'] = timestamp
    # File to store the data
    file_path = 'trucksuvidha_data.csv'
    # If file exists, append data. Otherwise, create and write data with header.
    if os.path.exists(file_path):
        df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        df.to_csv(file_path, mode='w', header=True, index=False)
    print(f"Data collected and saved at {timestamp}")

In [5]:

!pip install schedule==1.1.0

In [ ]:
import schedule
import time

# Schedule the function to run every 10 minutes
schedule.every(1/6).hours.do(collect_and_store_data)

while True:
  schedule.run_pending()
  time.sleep(1)

Data collected and saved at 2025/02/26_12:39:56
Data collected and saved at 2025/02/26_12:49:58
Data collected and saved at 2025/02/26_13:00:05


In [1]:
!pip install geopy


In [7]:
from geopy.geocoders import Nominatim
import pandas as pd

def get_city_data(city_name):
    
    #Gets the longitude, latitude, and population of a city using geopy and Nominatim.
    
    geolocator = Nominatim(user_agent="city_data_collector")
    location = geolocator.geocode(city_name)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        # Population data is not always available, so handle the exception
        try:
            population = location.raw['extratags']['population']
        except KeyError:
            population = "Not available"

        return latitude, longitude, population
    else:
        return None, None, None

# Assuming your DataFrame is named 'df' and has a 'Destination' column
city_data = []
for city in df['Destination City'].unique():  # Get unique destination cities
    latitude, longitude, population = get_city_data(city)
    if latitude and longitude:
        city_data.append([city, latitude, longitude, population])  # Include city name here

# Create a DataFrame and save to CSV
city_df = pd.DataFrame(city_data, columns=['City', 'Latitude', 'Longitude', 'Population'])  # Adjusted column names
city_df.to_csv('destination_data.csv', index=False)